In [41]:
import pandas as pd
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [57]:
# teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 
#          'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
#          'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
#          'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
#          'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
#          'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

# # create dictionary with info for each baller on an nba roster at the end of the 2020 season
# ballers = {'Name': [], 'Page': []}

# # iterate through each team
# for team in teams:
    
#     # set url for team's basketball reference page in 2020
#     url = 'https://www.basketball-reference.com/teams/'+team+'/2020.html'

#     # retrieve page with the requests module
#     html = requests.get(url)

#     # create BeautifulSoup object; parse with 'html.parser'
#     soup = bs(html.text, 'html.parser')

#     # scrape the name and bball ref page of each baller listed on the roster
#     for x in range(1,50):
#         try:
#             ballers['Name'].append(soup.find_all('tr')[x].a.text)
#             ballers['Page'].append('https://www.basketball-reference.com/' + soup.find_all('tr')[x].a['href'])
#         except:
#             # break loop when team's roster has been exhausted
#             break
            

# ballers['Page'] = np.unique(ballers['Page']).tolist()
# ballers['Name'] = np.unique(ballers['Name']).tolist()
# ogs = ballers['Name']

# # show ballers dictionary
# ballers

    

In [160]:
# create list with info for each baller on an nba roster at the end of the 2020 season
ballers = []

ogs = []

# iterate through each team
for team in teams:
    
    # set url for team's basketball reference page in 2020
    url = f'https://www.basketball-reference.com/teams/{team}/2020.html'

    # retrieve page with the requests module
    html = requests.get(url)

    # create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html.text, 'html.parser')

    # scrape the name and bball ref page of each baller listed on the roster
    for x in range(1,50):
        try:
            ballers.append({'Name': soup.find_all('tr')[x].a.text, 
                            'Page': f"https://www.basketball-reference.com{soup.find_all('tr')[x].a['href']}"})
            ogs.append(soup.find_all('tr')[x].a.text)
        except:
            # break loop when team's roster has been exhausted
            break

# show ballers dictionary
ballers

[{'Name': "DeAndre' Bembry",
  'Page': 'https://www.basketball-reference.com/players/b/bembrde01.html'},
 {'Name': 'Charlie Brown',
  'Page': 'https://www.basketball-reference.com/players/b/brownch02.html'},
 {'Name': 'Vince Carter',
  'Page': 'https://www.basketball-reference.com/players/c/cartevi01.html'},
 {'Name': 'John Collins',
  'Page': 'https://www.basketball-reference.com/players/c/collijo01.html'},
 {'Name': 'Allen Crabbe',
  'Page': 'https://www.basketball-reference.com/players/c/crabbal01.html'},
 {'Name': 'Dewayne Dedmon',
  'Page': 'https://www.basketball-reference.com/players/d/dedmode01.html'},
 {'Name': 'Bruno Fernando',
  'Page': 'https://www.basketball-reference.com/players/f/fernabr01.html'},
 {'Name': 'Brandon Goodwin',
  'Page': 'https://www.basketball-reference.com/players/g/goodwbr01.html'},
 {'Name': 'Treveon Graham',
  'Page': 'https://www.basketball-reference.com/players/g/grahatr01.html'},
 {'Name': 'Kevin Huerter',
  'Page': 'https://www.basketball-referenc

In [47]:
len(ballers['Page'])

530

In [161]:
nope = []

# go through each ballers bballref page
for i in range(0, len(ballers)):

    try:
        print(f"{i}: {ballers[i]['Name']}\n")

        # setup scrape
        url = ballers[i]['Page']
        html = requests.get(url)

        # go to page and scrape
        soup = bs(html.text, 'html.parser')

        # get conventional per game stats and put in pandas df
        stats = pd.read_html(url)[0]

        # filter for full seasons from 2015-2020
        standard = stats.loc[(stats['Season'] == '2019-20') | 
                  (stats['Season'] == '2018-19') |
                  (stats['Season'] == '2017-18') | 
                  (stats['Season'] == '2016-17') | 
                  (stats['Season'] == '2015-16') ]

        # get advanced per game stats and put in pandas df
        table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]
        stats = pd.read_html(table)[0]

        # filter for full seasons from 2015-2020 and remove redundant columns
        advanced = stats.loc[(stats['Season'] == '2019-20') | 
                  (stats['Season'] == '2018-19') |
                  (stats['Season'] == '2017-18') | 
                  (stats['Season'] == '2016-17') | 
                  (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

        # merge conventional and advanced per game stats into one pandas df
        stats = pd.merge(standard, advanced, on='Season')
        
        # get baller's photo, home city, and home country
        ballers[i]['Photo'] = soup.find_all('img', itemscope="image")[0]['src']
        ballers[i]['City'] = soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " ")
        ballers[i]['Country'] = soup.find_all('span')[13].text

        # place individual baller's stats in dictionary for all ballers
        for j in range(0, len(stats.columns)):

            # set stat being refered to as a variable
            stat = stats.columns[j]

            # most recent autobiographical/contextual data appended without manipulation
            if j < 5:
                ballers[i][stat] = stats[stat][len(stats[stat])-1]

            # games played/started are appended as an accumulative sum over last 5 years
            elif j < 7:
                stats[stat] = pd.to_numeric(stats[stat])
                ballers[i][stat] = stats[stat].sum()
                
            # per game stats are appended as an average over last 5 years
            else:
                stats[stat] = pd.to_numeric(stats[stat])
                ballers[i][stat] = stats[stat].mean()
        
        # remove unnecessary keys
        ballers[i].pop('Page')
        ballers[i].pop('Season')
        ballers[i].pop('Unnamed: 24')
        ballers[i].pop('Unnamed: 19')
    
    except:
        nope.append(ballers[i]['Name'])



# show ballers dictionary
ballers



   

0: DeAndre' Bembry

1: Charlie Brown

2: Vince Carter

3: John Collins

4: Allen Crabbe

5: Dewayne Dedmon

6: Bruno Fernando

7: Brandon Goodwin

8: Treveon Graham

9: Kevin Huerter

10: De'Andre Hunter

11: Damian Jones

12: Alex Len

13: Jabari Parker

14: Chandler Parsons

15: Cam Reddish

16: Jeff Teague

17: Evan Turner

18: Tyrone Wallace

19: Paul Watson

20: Trae Young

21: Jarrett Allen

22: Justin Anderson

23: Wilson Chandler

24: Chris Chiozza

25: Nicolas Claxton

26: Jamal Crawford

27: Spencer Dinwiddie

28: Henry Ellenson

29: Donta Hall

30: Joe Harris

31: Kyrie Irving

32: Tyler Johnson

33: DeAndre Jordan

34: Rodions Kurucs

35: Caris LeVert

36: Timothé Luwawu-Cabarrot

37: Jeremiah Martin

38: Džanan Musa

39: David Nwaba

40: Theo Pinson

41: Taurean Prince

42: Iman Shumpert

43: Garrett Temple

44: Lance Thomas

45: Jaylen Brown

46: Carsen Edwards

47: Tacko Fall

48: Javonte Green

49: Gordon Hayward

50: Enes Kanter

51: Romeo Langford

52: Semi Ojeleye

5

411: Isaiah Roby

412: Dennis Schröder

413: Al-Farouq Aminu

414: D.J. Augustin

415: Mohamed Bamba

416: Khem Birch

417: Michael Carter-Williams

418: Gary Clark

419: James Ennis

420: Evan Fournier

421: Melvin Frazier

422: Markelle Fultz

423: Aaron Gordon

424: Jonathan Isaac

425: Wesley Iwundu

426: Amile Jefferson

427: B.J. Johnson

428: Vic Law

429: Josh Magette

430: Terrence Ross

431: Nikola Vučević

432: Jonah Bolden

433: Trey Burke

434: Alec Burks

435: Joel Embiid

436: James Ennis

437: Tobias Harris

438: Al Horford

439: Furkan Korkmaz

440: Shake Milton

441: Raul Neto

442: Kyle O'Quinn

443: Norvel Pelle

444: Josh Richardson

445: Glenn Robinson

446: Mike Scott

447: Marial Shayok

448: Ben Simmons

449: Zhaire Smith

450: Matisse Thybulle

451: Deandre Ayton

452: Aron Baynes

453: Jonah Bolden

454: Devin Booker

455: Mikal Bridges

456: Jevon Carter

457: Cheick Diallo

458: Jared Harper

459: Ty Jerome

460: Cameron Johnson

461: Tyler Johnson

462: Fr

[{'Name': "DeAndre' Bembry",
  'Photo': 'https://www.basketball-reference.com/req/202104203/images/players/bembrde01.jpg',
  'City': 'Charlotte, North Carolina',
  'Country': 'us',
  'Age': 25.0,
  'Tm': 'ATL',
  'Lg': 'NBA',
  'Pos': 'SG',
  'G': 189.0,
  'GS': 23.0,
  'MP': 18.025,
  'FG': 2.25,
  'FGA': 5.075,
  'FG%': 0.44899999999999995,
  '3P': 0.325,
  '3PA': 1.325,
  '3P%': 0.23575,
  '2P': 1.9,
  '2PA': 3.7750000000000004,
  '2P%': 0.5145,
  'eFG%': 0.47950000000000004,
  'FT': 0.65,
  'FTA': 1.125,
  'FT%': 0.53325,
  'ORB': 0.55,
  'DRB': 2.5250000000000004,
  'TRB': 3.075,
  'AST': 1.75,
  'STL': 0.8999999999999999,
  'BLK': 0.375,
  'TOV': 1.3250000000000002,
  'PF': 1.6,
  'PTS': 5.525,
  'PER': 9.2,
  'TS%': 0.49150000000000005,
  '3PAr': 0.246,
  'FTr': 0.21325,
  'ORB%': 3.275,
  'DRB%': 14.95,
  'TRB%': 9.025,
  'AST%': 13.599999999999998,
  'STL%': 2.175,
  'BLK%': 1.75,
  'TOV%': 18.7,
  'USG%': 15.899999999999999,
  'OWS': -0.6,
  'DWS': 0.8500000000000001,
  'WS':

In [162]:
nope

['Sviatoslav Mykhailiuk', 'Dusty Hannahs', 'Chimezie Metu']

In [163]:
for baller in ballers:
    for no in nope:
        if baller['Name'] == no:
            ballers.remove(baller)



In [164]:
for baller in ballers:
    if baller['Name'] == 'Dusty Hannahs':
        print(baller['Name'])

In [165]:
for baller in ballers:
    if len(baller.keys()) != 51:
        print(f"{baller['Name']}: {len(baller.keys())}")

DeAndre' Bembry: 53
Charlie Brown: 53
Vince Carter: 53
John Collins: 53
Allen Crabbe: 53
Dewayne Dedmon: 53
Bruno Fernando: 53
Brandon Goodwin: 53
Treveon Graham: 53
Kevin Huerter: 53
De'Andre Hunter: 53
Damian Jones: 53
Alex Len: 53
Jabari Parker: 53
Chandler Parsons: 53
Cam Reddish: 53
Jeff Teague: 53
Evan Turner: 53
Tyrone Wallace: 53
Paul Watson: 53
Trae Young: 53
Jarrett Allen: 53
Justin Anderson: 53
Wilson Chandler: 53
Chris Chiozza: 53
Nicolas Claxton: 53
Jamal Crawford: 53
Spencer Dinwiddie: 53
Henry Ellenson: 53
Donta Hall: 53
Joe Harris: 53
Kyrie Irving: 53
Tyler Johnson: 53
DeAndre Jordan: 53
Rodions Kurucs: 53
Caris LeVert: 53
Timothé Luwawu-Cabarrot: 53
Jeremiah Martin: 53
Džanan Musa: 53
David Nwaba: 53
Theo Pinson: 53
Taurean Prince: 53
Iman Shumpert: 53
Garrett Temple: 53
Lance Thomas: 53
Jaylen Brown: 53
Carsen Edwards: 53
Tacko Fall: 53
Javonte Green: 53
Gordon Hayward: 53
Enes Kanter: 53
Romeo Langford: 53
Semi Ojeleye: 53
Vincent Poirier: 53
Marcus Smart: 53
Jayson 

In [166]:
stat_arrays = {}

for stat in ballers[0].keys():
    stat_arrays[stat] = []
    
stat_arrays

{'Name': [],
 'Photo': [],
 'City': [],
 'Country': [],
 'Age': [],
 'Tm': [],
 'Lg': [],
 'Pos': [],
 'G': [],
 'GS': [],
 'MP': [],
 'FG': [],
 'FGA': [],
 'FG%': [],
 '3P': [],
 '3PA': [],
 '3P%': [],
 '2P': [],
 '2PA': [],
 '2P%': [],
 'eFG%': [],
 'FT': [],
 'FTA': [],
 'FT%': [],
 'ORB': [],
 'DRB': [],
 'TRB': [],
 'AST': [],
 'STL': [],
 'BLK': [],
 'TOV': [],
 'PF': [],
 'PTS': [],
 'PER': [],
 'TS%': [],
 '3PAr': [],
 'FTr': [],
 'ORB%': [],
 'DRB%': [],
 'TRB%': [],
 'AST%': [],
 'STL%': [],
 'BLK%': [],
 'TOV%': [],
 'USG%': [],
 'OWS': [],
 'DWS': [],
 'WS': [],
 'WS/48': [],
 'OBPM': [],
 'DBPM': [],
 'BPM': [],
 'VORP': []}

In [167]:
for baller in ballers:
    for stat in ballers[0].keys():
        stat_arrays[stat].append(baller[stat])
        


In [168]:
stat_arrays

{'Name': ["DeAndre' Bembry",
  'Charlie Brown',
  'Vince Carter',
  'John Collins',
  'Allen Crabbe',
  'Dewayne Dedmon',
  'Bruno Fernando',
  'Brandon Goodwin',
  'Treveon Graham',
  'Kevin Huerter',
  "De'Andre Hunter",
  'Damian Jones',
  'Alex Len',
  'Jabari Parker',
  'Chandler Parsons',
  'Cam Reddish',
  'Jeff Teague',
  'Evan Turner',
  'Tyrone Wallace',
  'Paul Watson',
  'Trae Young',
  'Jarrett Allen',
  'Justin Anderson',
  'Wilson Chandler',
  'Chris Chiozza',
  'Nicolas Claxton',
  'Jamal Crawford',
  'Spencer Dinwiddie',
  'Henry Ellenson',
  'Donta Hall',
  'Joe Harris',
  'Kyrie Irving',
  'Tyler Johnson',
  'DeAndre Jordan',
  'Rodions Kurucs',
  'Caris LeVert',
  'Timothé Luwawu-Cabarrot',
  'Jeremiah Martin',
  'Džanan Musa',
  'David Nwaba',
  'Theo Pinson',
  'Taurean Prince',
  'Iman Shumpert',
  'Garrett Temple',
  'Lance Thomas',
  'Jaylen Brown',
  'Carsen Edwards',
  'Tacko Fall',
  'Javonte Green',
  'Gordon Hayward',
  'Enes Kanter',
  'Romeo Langford',
 

In [169]:
for stat in stat_arrays.keys():
    print(f'{stat}: {len(stat_arrays[stat])}')

Name: 589
Photo: 589
City: 589
Country: 589
Age: 589
Tm: 589
Lg: 589
Pos: 589
G: 589
GS: 589
MP: 589
FG: 589
FGA: 589
FG%: 589
3P: 589
3PA: 589
3P%: 589
2P: 589
2PA: 589
2P%: 589
eFG%: 589
FT: 589
FTA: 589
FT%: 589
ORB: 589
DRB: 589
TRB: 589
AST: 589
STL: 589
BLK: 589
TOV: 589
PF: 589
PTS: 589
PER: 589
TS%: 589
3PAr: 589
FTr: 589
ORB%: 589
DRB%: 589
TRB%: 589
AST%: 589
STL%: 589
BLK%: 589
TOV%: 589
USG%: 589
OWS: 589
DWS: 589
WS: 589
WS/48: 589
OBPM: 589
DBPM: 589
BPM: 589
VORP: 589


In [170]:
for i in range(0, len(baller_df.columns)):
    print(f'{i}: {baller_df.columns[i]}, {baller_df[baller_df.columns[i]].dtypes}')

0: Name, object
1: Age, float64
2: Tm, object
3: Lg, object
4: Pos, object
5: G, object
6: GS, object
7: MP, float64
8: FG, float64
9: FGA, float64
10: FG%, float64
11: 3P, float64
12: 3PA, float64
13: 3P%, float64
14: 2P, float64
15: 2PA, float64
16: 2P%, float64
17: eFG%, float64
18: FT, float64
19: FTA, float64
20: FT%, float64
21: ORB, float64
22: DRB, float64
23: TRB, float64
24: AST, float64
25: STL, float64
26: BLK, float64
27: TOV, float64
28: PF, float64
29: PTS, float64
30: PER, float64
31: TS%, float64
32: 3PAr, float64
33: FTr, float64
34: ORB%, float64
35: DRB%, float64
36: TRB%, float64
37: AST%, float64
38: STL%, float64
39: BLK%, float64
40: TOV%, float64
41: USG%, float64
42: OWS, float64
43: DWS, float64
44: WS, float64
45: WS/48, float64
46: OBPM, float64
47: DBPM, float64
48: BPM, float64
49: VORP, float64
50: Photo, object
51: City, object
52: Country, object


In [171]:
baller_df = pd.DataFrame(stat_arrays)
baller_df

,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,25.0,ATL,NBA,SG,189.0,23.0,...,18.700000,15.900000,-0.600000,0.850000,0.250000,0.008000,-3.875000,0.575000,-3.275000,-0.175000
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",us,22.0,ATL,NBA,SG,10.0,0.0,...,12.400000,25.100000,0.000000,0.000000,0.000000,-0.028000,-1.100000,-1.400000,-2.500000,0.000000
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",us,43.0,ATL,NBA,PF,327.0,32.0,...,9.160000,16.280000,0.760000,0.980000,1.740000,0.061000,-1.220000,-0.060000,-1.280000,0.360000
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",us,22.0,ATL,NBA,PF,176.0,126.0,...,11.866667,21.433333,4.066667,1.400000,5.433333,0.159000,2.033333,-0.766667,1.266667,1.266667
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",us,27.0,MIN,NBA,SF,500.0,109.0,...,9.246154,13.876923,0.484615,0.423077,0.915385,0.007231,-3.446154,-1.476923,-4.900000,-0.146154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,31.0,WAS,NBA,PG,749.0,408.0,...,13.630769,22.284615,0.369231,1.053846,1.453846,0.047462,-0.230769,-0.800000,-1.061538,0.300000
585,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,30.0,WAS,NBA,PG,402.0,282.0,...,15.369231,29.023077,1.323077,0.453846,1.753846,0.031615,-0.015385,-3.323077,-3.353846,0.423077
586,Jarrod Uthoff,https://www.basketball-reference.com/req/20210...,"Cedar Rapids, Iowa",us,26.0,WAS,NBA,PF,51.0,0.0,...,0.680000,16.780000,0.010000,0.010000,0.050000,-0.011200,-3.940000,-2.080000,-5.980000,0.000000
587,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",de,22.0,WAS,NBA,C,88.0,10.0,...,17.200000,20.150000,0.400000,0.550000,0.950000,0.059500,-2.950000,-0.350000,-3.300000,-0.200000


In [172]:
ballers_percentile = {}
ballers_rank = {}



        
for i in range(0, len(baller_df.columns)):
    if (i == 0) | (i == 1) | (i == 2) | (i == 3) |(i == 5) | (i == 6) |(i == 7):
        ballers_percentile[baller_df.columns[i]] = []
        ballers_rank[baller_df.columns[i]] = []
    else:
        ballers_percentile['%ile in ' + baller_df.columns[i]] = []
        ballers_rank['Rank in ' + baller_df.columns[i]] = []

        


for i in range(0, len(baller_df.columns)):

    stat = baller_df.columns[i]

    for j in range(0, len(baller_df[stat])):

        if (i == 0) | (i == 1) | (i == 2) | (i == 3) |(i == 5) | (i == 6) |(i == 7):

            ballers_percentile[stat].append(baller_df[stat][j])
            ballers_rank[stat].append(baller_df[stat][j])

        else:

            ranked = baller_df.sort_values(stat)[stat].to_numpy()

            for k in range(0, len(ranked)):
                if ranked[k] == baller_df[stat][j]:
    #                 print([baller_df['Name'][j], stat, ranked[k], k])
                    below = k
                    break

            percentile = round((below/len(ranked)*100), 2)
            rank = len(ranked) - below
#             print([baller_df['Name'][j], rank, stat])
            
#                 print([baller_df['Name'][j], [rank,len(ranked)], percentile, stat])
            ballers_percentile['%ile in ' + stat].append(percentile)
            ballers_rank['Rank in ' + stat].append(rank)


ballers_percentile.pop('City')
ballers_percentile.pop('Country')
ballers_percentile.pop('Photo')
ballers_percentile.pop('Tm')
ballers_percentile.pop('Lg')
ballers_percentile.pop('Pos')
ballers_percentile


ballers_rank.pop('City')
ballers_rank.pop('Country')
ballers_rank.pop('Photo')
ballers_rank.pop('Tm')
ballers_rank.pop('Lg')
ballers_rank.pop('Pos')
ballers_rank


{'Name': ["DeAndre' Bembry",
  'Charlie Brown',
  'Vince Carter',
  'John Collins',
  'Allen Crabbe',
  'Dewayne Dedmon',
  'Bruno Fernando',
  'Brandon Goodwin',
  'Treveon Graham',
  'Kevin Huerter',
  "De'Andre Hunter",
  'Damian Jones',
  'Alex Len',
  'Jabari Parker',
  'Chandler Parsons',
  'Cam Reddish',
  'Jeff Teague',
  'Evan Turner',
  'Tyrone Wallace',
  'Paul Watson',
  'Trae Young',
  'Jarrett Allen',
  'Justin Anderson',
  'Wilson Chandler',
  'Chris Chiozza',
  'Nicolas Claxton',
  'Jamal Crawford',
  'Spencer Dinwiddie',
  'Henry Ellenson',
  'Donta Hall',
  'Joe Harris',
  'Kyrie Irving',
  'Tyler Johnson',
  'DeAndre Jordan',
  'Rodions Kurucs',
  'Caris LeVert',
  'Timothé Luwawu-Cabarrot',
  'Jeremiah Martin',
  'Džanan Musa',
  'David Nwaba',
  'Theo Pinson',
  'Taurean Prince',
  'Iman Shumpert',
  'Garrett Temple',
  'Lance Thomas',
  'Jaylen Brown',
  'Carsen Edwards',
  'Tacko Fall',
  'Javonte Green',
  'Gordon Hayward',
  'Enes Kanter',
  'Romeo Langford',
 

In [173]:
ballers_percentile

{'Name': ["DeAndre' Bembry",
  'Charlie Brown',
  'Vince Carter',
  'John Collins',
  'Allen Crabbe',
  'Dewayne Dedmon',
  'Bruno Fernando',
  'Brandon Goodwin',
  'Treveon Graham',
  'Kevin Huerter',
  "De'Andre Hunter",
  'Damian Jones',
  'Alex Len',
  'Jabari Parker',
  'Chandler Parsons',
  'Cam Reddish',
  'Jeff Teague',
  'Evan Turner',
  'Tyrone Wallace',
  'Paul Watson',
  'Trae Young',
  'Jarrett Allen',
  'Justin Anderson',
  'Wilson Chandler',
  'Chris Chiozza',
  'Nicolas Claxton',
  'Jamal Crawford',
  'Spencer Dinwiddie',
  'Henry Ellenson',
  'Donta Hall',
  'Joe Harris',
  'Kyrie Irving',
  'Tyler Johnson',
  'DeAndre Jordan',
  'Rodions Kurucs',
  'Caris LeVert',
  'Timothé Luwawu-Cabarrot',
  'Jeremiah Martin',
  'Džanan Musa',
  'David Nwaba',
  'Theo Pinson',
  'Taurean Prince',
  'Iman Shumpert',
  'Garrett Temple',
  'Lance Thomas',
  'Jaylen Brown',
  'Carsen Edwards',
  'Tacko Fall',
  'Javonte Green',
  'Gordon Hayward',
  'Enes Kanter',
  'Romeo Langford',
 

In [174]:
percentile_df = pd.DataFrame(ballers_percentile)
rank_df = pd.DataFrame(ballers_rank)

all_stats = pd.merge(baller_df, rank_df, on='Name')
all_stats = pd.merge(all_stats, percentile_df, on='Name')
all_stats


,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,%ile in TOV%,%ile in USG%,%ile in OWS,%ile in DWS,%ile in WS,%ile in WS/48,%ile in OBPM,%ile in DBPM,%ile in BPM,%ile in VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,25.0,ATL,NBA,SG,189.0,23.0,...,91.68,32.09,2.38,54.50,22.92,13.75,16.30,75.04,26.83,13.07
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",us,22.0,ATL,NBA,SG,10.0,0.0,...,51.44,89.64,19.35,0.17,6.45,6.28,54.84,14.94,34.63,32.26
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",us,43.0,ATL,NBA,PF,327.0,32.0,...,16.47,34.63,56.71,60.95,58.40,39.56,52.80,55.86,54.67,65.70
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",us,22.0,ATL,NBA,PF,176.0,126.0,...,44.48,75.38,94.91,74.36,92.70,90.15,92.02,30.90,86.93,88.46
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",us,27.0,MIN,NBA,SF,500.0,109.0,...,16.98,15.28,48.22,33.28,38.71,13.41,19.02,13.41,13.58,14.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Admiral Schofield,https://www.basketball-reference.com/req/20210...,"London, United Kingdom",gb,22.0,WAS,NBA,PF,33.0,2.0,...,4.58,6.28,28.18,9.68,13.92,18.00,12.56,19.19,11.21,7.30
981,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,31.0,WAS,NBA,PG,749.0,408.0,...,63.67,79.46,41.60,63.84,52.97,30.22,73.34,29.03,58.57,61.29
982,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,30.0,WAS,NBA,PG,402.0,282.0,...,76.74,96.26,73.17,34.30,58.91,22.92,76.23,1.87,26.32,67.23
983,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",de,22.0,WAS,NBA,C,88.0,10.0,...,86.59,67.74,43.80,38.37,39.56,39.05,27.16,46.86,26.49,10.36


In [181]:
all_stats.drop_duplicates("Name", inplace=True)

all_stats

,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,%ile in TOV%,%ile in USG%,%ile in OWS,%ile in DWS,%ile in WS,%ile in WS/48,%ile in OBPM,%ile in DBPM,%ile in BPM,%ile in VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,25.0,ATL,NBA,SG,189.0,23.0,...,91.68,32.09,2.38,54.50,22.92,13.75,16.30,75.04,26.83,13.07
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",us,22.0,ATL,NBA,SG,10.0,0.0,...,51.44,89.64,19.35,0.17,6.45,6.28,54.84,14.94,34.63,32.26
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",us,43.0,ATL,NBA,PF,327.0,32.0,...,16.47,34.63,56.71,60.95,58.40,39.56,52.80,55.86,54.67,65.70
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",us,22.0,ATL,NBA,PF,176.0,126.0,...,44.48,75.38,94.91,74.36,92.70,90.15,92.02,30.90,86.93,88.46
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",us,27.0,MIN,NBA,SF,500.0,109.0,...,16.98,15.28,48.22,33.28,38.71,13.41,19.02,13.41,13.58,14.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Admiral Schofield,https://www.basketball-reference.com/req/20210...,"London, United Kingdom",gb,22.0,WAS,NBA,PF,33.0,2.0,...,4.58,6.28,28.18,9.68,13.92,18.00,12.56,19.19,11.21,7.30
981,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,31.0,WAS,NBA,PG,749.0,408.0,...,63.67,79.46,41.60,63.84,52.97,30.22,73.34,29.03,58.57,61.29
982,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,30.0,WAS,NBA,PG,402.0,282.0,...,76.74,96.26,73.17,34.30,58.91,22.92,76.23,1.87,26.32,67.23
983,Moritz Wagner,https://www.basketball-reference.com/req/20210...,"Berlin, Germany",de,22.0,WAS,NBA,C,88.0,10.0,...,86.59,67.74,43.80,38.37,39.56,39.05,27.16,46.86,26.49,10.36


In [187]:
all_american = all_stats.loc[all_stats['Country'] == 'us', :]


,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,%ile in TOV%,%ile in USG%,%ile in OWS,%ile in DWS,%ile in WS,%ile in WS/48,%ile in OBPM,%ile in DBPM,%ile in BPM,%ile in VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,25.0,ATL,NBA,SG,189.0,23.0,...,91.68,32.09,2.38,54.50,22.92,13.75,16.30,75.04,26.83,13.07
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",us,22.0,ATL,NBA,SG,10.0,0.0,...,51.44,89.64,19.35,0.17,6.45,6.28,54.84,14.94,34.63,32.26
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",us,43.0,ATL,NBA,PF,327.0,32.0,...,16.47,34.63,56.71,60.95,58.40,39.56,52.80,55.86,54.67,65.70
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",us,22.0,ATL,NBA,PF,176.0,126.0,...,44.48,75.38,94.91,74.36,92.70,90.15,92.02,30.90,86.93,88.46
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",us,27.0,MIN,NBA,SF,500.0,109.0,...,16.98,15.28,48.22,33.28,38.71,13.41,19.02,13.41,13.58,14.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,Gary Payton,https://www.basketball-reference.com/req/20210...,"Seattle, Washington",us,27.0,WAS,NBA,SF,176.0,53.0,...,33.11,42.95,18.51,16.98,17.15,38.20,31.75,77.25,42.61,28.01
979,Justin Robinson,https://www.basketball-reference.com/req/20210...,"Manassas, Virginia",us,22.0,WAS,NBA,PG,9.0,0.0,...,94.23,9.34,19.35,0.17,6.45,13.07,33.45,2.04,10.02,32.26
981,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,31.0,WAS,NBA,PG,749.0,408.0,...,63.67,79.46,41.60,63.84,52.97,30.22,73.34,29.03,58.57,61.29
982,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,30.0,WAS,NBA,PG,402.0,282.0,...,76.74,96.26,73.17,34.30,58.91,22.92,76.23,1.87,26.32,67.23


In [192]:
# THIS IS IT!
all_american

,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,%ile in TOV%,%ile in USG%,%ile in OWS,%ile in DWS,%ile in WS,%ile in WS/48,%ile in OBPM,%ile in DBPM,%ile in BPM,%ile in VORP
0,DeAndre' Bembry,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,25.0,ATL,NBA,SG,189.0,23.0,...,91.68,32.09,2.38,54.50,22.92,13.75,16.30,75.04,26.83,13.07
1,Charlie Brown,https://www.basketball-reference.com/req/20210...,"Philadelphia, Pennsylvania",us,22.0,ATL,NBA,SG,10.0,0.0,...,51.44,89.64,19.35,0.17,6.45,6.28,54.84,14.94,34.63,32.26
2,Vince Carter,https://www.basketball-reference.com/req/20210...,"Daytona Beach, Florida",us,43.0,ATL,NBA,PF,327.0,32.0,...,16.47,34.63,56.71,60.95,58.40,39.56,52.80,55.86,54.67,65.70
3,John Collins,https://www.basketball-reference.com/req/20210...,"Layton, Utah",us,22.0,ATL,NBA,PF,176.0,126.0,...,44.48,75.38,94.91,74.36,92.70,90.15,92.02,30.90,86.93,88.46
4,Allen Crabbe,https://www.basketball-reference.com/req/20210...,"Los Angeles, California",us,27.0,MIN,NBA,SF,500.0,109.0,...,16.98,15.28,48.22,33.28,38.71,13.41,19.02,13.41,13.58,14.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,Gary Payton,https://www.basketball-reference.com/req/20210...,"Seattle, Washington",us,27.0,WAS,NBA,SF,176.0,53.0,...,33.11,42.95,18.51,16.98,17.15,38.20,31.75,77.25,42.61,28.01
979,Justin Robinson,https://www.basketball-reference.com/req/20210...,"Manassas, Virginia",us,22.0,WAS,NBA,PG,9.0,0.0,...,94.23,9.34,19.35,0.17,6.45,13.07,33.45,2.04,10.02,32.26
981,Ish Smith,https://www.basketball-reference.com/req/20210...,"Charlotte, North Carolina",us,31.0,WAS,NBA,PG,749.0,408.0,...,63.67,79.46,41.60,63.84,52.97,30.22,73.34,29.03,58.57,61.29
982,Isaiah Thomas,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,30.0,WAS,NBA,PG,402.0,282.0,...,76.74,96.26,73.17,34.30,58.91,22.92,76.23,1.87,26.32,67.23


In [56]:
# nope = []

# ballers['Photo'] = [] 
# ballers['City'] = []
# ballers['Country'] = []

# # go through each ballers bballref page
# for i in range(0, (len(ballers['Page'])-1)):

#     try:
#         print(f"{i}: {ballers['Name'][i]}\n")

#         # setup scrape
#         url = ballers['Page'][i]
#         html = requests.get(url)

#         # go to page and scrape
#         soup = bs(html.text, 'html.parser')

#         # get conventional per game stats and put in pandas df
#         stats = pd.read_html(url)[0]

#         # filter for full seasons from 2015-2020
#         standard = stats.loc[(stats['Season'] == '2019-20') | 
#                   (stats['Season'] == '2018-19') |
#                   (stats['Season'] == '2017-18') | 
#                   (stats['Season'] == '2016-17') | 
#                   (stats['Season'] == '2015-16') ]

#         # get advanced per game stats and put in pandas df
#         table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]
#         stats = pd.read_html(table)[0]

#         # filter for full seasons from 2015-2020 and remove redundant columns
#         advanced = stats.loc[(stats['Season'] == '2019-20') | 
#                   (stats['Season'] == '2018-19') |
#                   (stats['Season'] == '2017-18') | 
#                   (stats['Season'] == '2016-17') | 
#                   (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

#         # merge conventional and advanced per game stats into one pandas df
#         stats = pd.merge(standard, advanced, on='Season')

#         # create new keys in "ballers" dictionary for each statistical category
#         if i == 0:
#             for c in stats.columns:
#                     ballers[c] = []

#         # place individual baller's stats in dictionary for all ballers
#         for j in range(0, len(stats.columns)):

#             # set stat being refered to as a variable
#             stat = stats.columns[j]

#             # most recent autobiographical/contextual data appended without manipulation
#             if j < 5:
#                 ballers[stat].append(stats[stat][len(stats[stat])-1])
                

#             # games played/started are appended as an accumulative sum over last 5 years
#             elif j < 7:
#                 ballers[stat].append(stats[stat].sum())
                

#             # per game stats are appended as an average over last 5 years
#             else:
#                 stats[stat] = pd.to_numeric(stats[stat])
# #                 print(stat + ': ')
# #                 print(str(stats[stat].mean()))
#                 ballers[stat].append(stats[stat].mean())

#         # get baller's photo, home city, and home country
#         ballers['Photo'].append(soup.find_all('img', itemscope="image")[0]['src'])
#         ballers['City'].append(soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " "))
#         ballers['Country'].append(soup.find_all('span')[13].text)
    
#     except:
#         nope.append(ballers['Name'][i])
# #         ballers['Name'].remove(ballers['Name'][i])
# #         ballers['Page'].remove(ballers['Page'][i])

# # # remove unnecessary keys
# # ballers.pop('Page')
# # ballers.pop('Season')
# # ballers.pop('Unnamed: 24')
# # ballers.pop('Unnamed: 19')

# # show ballers dictionary
# ballers


In [55]:
for mmm in ballers.keys():
    print(mmm + ": " + str(len(ballers[mmm])))


# for baller in ballers["Name"]:
#     if baller == 'Chimezie Metu':
#         print(baller)

# import numpy as np
# diff = np.setdiff1d(ogs,ballers["Name"])
# diff

    
    

Name: 530
Page: 530
Photo: 526
City: 526
Country: 526
Season: 526
Age: 526
Tm: 526
Lg: 526
Pos: 526
G: 526
GS: 526
MP: 526
FG: 526
FGA: 526
FG%: 526
3P: 526
3PA: 526
3P%: 526
2P: 526
2PA: 526
2P%: 526
eFG%: 526
FT: 526
FTA: 526
FT%: 526
ORB: 526
DRB: 526
TRB: 526
AST: 526
STL: 526
BLK: 526
TOV: 526
PF: 526
PTS: 526
PER: 526
TS%: 526
3PAr: 526
FTr: 526
ORB%: 526
DRB%: 526
TRB%: 526
AST%: 526
STL%: 526
BLK%: 526
TOV%: 526
USG%: 526
Unnamed: 19: 526
OWS: 526
DWS: 526
WS: 526
WS/48: 526
Unnamed: 24: 526
OBPM: 526
DBPM: 526
BPM: 526
VORP: 526


In [53]:
nope

['Isaiah Roby', 'Luka Dončić', 'Matthew Dellavedova']

In [54]:
len(ballers['PTS'])

526

In [323]:
nope

['Sviatoslav Mykhailiuk', 'Dusty Hannahs', 'Chimezie Metu']

In [335]:
this = pd.DataFrame({"Name": ballers["Name"], "Page": ballers["Page"]})

that = this["Name"].value_counts()

for baller in that:
    print(baller)


3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [317]:
ayy = [0,1,2,3]

ayy.remove(0)

len(ayy)

3

In [9]:
# Try to scrape the site using the URL
url = 'https://www.basketball-reference.com/teams/LAL/2020.html'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get article title and summary paragraph

ballers = []

for x in range(1,50):
    try:
        ballers.append({'Name': soup.find_all('tr')[x].a.text, 
                        'Page': 'https://www.basketball-reference.com/' + soup.find_all('tr')[x].a['href']})
    except:
        break

ballers


[{'Name': 'Kostas Antetokounmpo',
  'Page': 'https://www.basketball-reference.com//players/a/antetko01.html'},
 {'Name': 'Avery Bradley',
  'Page': 'https://www.basketball-reference.com//players/b/bradlav01.html'},
 {'Name': 'Devontae Cacok',
  'Page': 'https://www.basketball-reference.com//players/c/cacokde01.html'},
 {'Name': 'Kentavious Caldwell-Pope',
  'Page': 'https://www.basketball-reference.com//players/c/caldwke01.html'},
 {'Name': 'Alex Caruso',
  'Page': 'https://www.basketball-reference.com//players/c/carusal01.html'},
 {'Name': 'Quinn Cook',
  'Page': 'https://www.basketball-reference.com//players/c/cookqu01.html'},
 {'Name': 'Troy Daniels',
  'Page': 'https://www.basketball-reference.com//players/d/danietr01.html'},
 {'Name': 'Anthony Davis',
  'Page': 'https://www.basketball-reference.com//players/d/davisan02.html'},
 {'Name': 'Jared Dudley',
  'Page': 'https://www.basketball-reference.com//players/d/dudleja01.html'},
 {'Name': 'Danny Green',
  'Page': 'https://www.baske

In [13]:
for i in range(0, len(ballers)-1):
    
    # setup scrape
    url = ballers[i]['Page']
    html = requests.get(url)
    

    # scrape
    soup = bs(html.text, 'html.parser')
    
    
    stats = pd.read_html(url)[0]

    standard = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ]
    
    table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]
    stats = pd.read_html(table)[0]

    advanced = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

    stats = pd.merge(standard, advanced, on='Season')
  

    for j in range(0, len(stats.columns)):
        stat = stats.columns[j]
        if j < 5:
            ballers[i][stat] = stats[stat][len(stats[stat])-1]
        elif j < 7:
            ballers[i][stat] = stats[stat].sum()
        else:
            stats[stat] = pd.to_numeric(stats[stat])
            ballers[i][stat] = stats[stat].mean()
            
#     ballers['Photo'].append(soup.find_all('img', alt=("Photo of " + ballers['Name'][i]))[0]['src'])
#     ballers['City'].append(soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " "))
#     ballers['Country'].append(soup.find_all('span')[13].text)



# ballers.pop('Page')
# ballers.pop('Season')
# ballers.pop('Unnamed: 24')
# ballers.pop('Unnamed: 19')
# ballers


In [14]:
ballers

[{'Name': 'Kostas Antetokounmpo',
  'Page': 'https://www.basketball-reference.com//players/a/antetko01.html',
  'Season': '2019-20',
  'Age': 22.0,
  'Tm': 'LAL',
  'Lg': 'NBA',
  'Pos': 'PF',
  'G': 7.0,
  'GS': 0.0,
  'MP': 4.75,
  'FG': 0.3,
  'FGA': 1.05,
  'FG%': 0.5,
  '3P': 0.0,
  '3PA': 0.0,
  '3P%': nan,
  '2P': 0.3,
  '2PA': 1.05,
  '2P%': 0.5,
  'eFG%': 0.5,
  'FT': 0.6,
  'FTA': 1.2,
  'FT%': 0.5,
  'ORB': 0.2,
  'DRB': 0.35,
  'TRB': 0.55,
  'AST': 0.2,
  'STL': 0.5,
  'BLK': 0.0,
  'TOV': 0.35,
  'PF': 0.2,
  'PTS': 1.2,
  'PER': 7.7,
  'TS%': 0.556,
  '3PAr': 0.0,
  'FTr': 1.0,
  'ORB%': 5.55,
  'DRB%': 7.55,
  'TRB%': 6.55,
  'AST%': 6.85,
  'STL%': 4.4,
  'BLK%': 0.0,
  'TOV%': 18.95,
  'USG%': 16.4,
  'Unnamed: 19': nan,
  'OWS': 0.0,
  'DWS': 0.0,
  'WS': 0.0,
  'WS/48': -0.026999999999999996,
  'Unnamed: 24': nan,
  'OBPM': -6.2,
  'DBPM': -0.5,
  'BPM': -6.699999999999999,
  'VORP': 0.0},
 {'Name': 'Avery Bradley',
  'Page': 'https://www.basketball-reference.com//p

In [1]:
teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 
         'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
         'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
         'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
         'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
         'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

for team in teams:
    team_url = 'https://www.basketball-reference.com/teams/'+team+'/2020.html'
#     print(team+':')
#     print(pd.read_html(team_url)[0])
    

In [4]:
# Try to scrape the site using the URL
url = 'https://www.basketball-reference.com/teams/LAL/2020.html'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get article title and summary paragraph

ballers = {'Name': [], 'Page': [], 'Photo': [], 'City': [], 'Country': []}

for x in range(1,50):
    try:
        ballers['Name'].append(soup.find_all('tr')[x].a.text)
        ballers['Page'].append('https://www.basketball-reference.com/' + soup.find_all('tr')[x].a['href'])
    except:
        break

ballers


{'Name': ['Kostas Antetokounmpo',
  'Avery Bradley',
  'Devontae Cacok',
  'Kentavious Caldwell-Pope',
  'Alex Caruso',
  'Quinn Cook',
  'Troy Daniels',
  'Anthony Davis',
  'Jared Dudley',
  'Danny Green',
  'Talen Horton-Tucker',
  'Dwight Howard',
  'LeBron James',
  'Kyle Kuzma',
  'JaVale McGee',
  'Markieff Morris',
  'Zach Norvell',
  'Rajon Rondo',
  'J.R. Smith',
  'Dion Waiters'],
 'Page': ['https://www.basketball-reference.com//players/a/antetko01.html',
  'https://www.basketball-reference.com//players/b/bradlav01.html',
  'https://www.basketball-reference.com//players/c/cacokde01.html',
  'https://www.basketball-reference.com//players/c/caldwke01.html',
  'https://www.basketball-reference.com//players/c/carusal01.html',
  'https://www.basketball-reference.com//players/c/cookqu01.html',
  'https://www.basketball-reference.com//players/d/danietr01.html',
  'https://www.basketball-reference.com//players/d/davisan02.html',
  'https://www.basketball-reference.com//players/d/dud

In [7]:
for i in range(0, len(ballers['Page'])):
    
    # setup scrape
    url = ballers['Page'][i]
    html = requests.get(url)

    # scrape
    soup = bs(html.text, 'html.parser')
    table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]

    stats = pd.read_html(table)[0]

    advanced = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

    stats = pd.read_html(url)[0]

    standard = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ]

    stats = pd.merge(standard, advanced, on='Season')
    
    if i == 0:
        for c in stats.columns:
                ballers[c] = []    

    for j in range(0, len(stats.columns)):
        stat = stats.columns[j]
        if j < 5:
            ballers[stat].append(stats[stat][len(stats[stat])-1])
        elif j < 7:
            ballers[stat].append(stats[stat].sum())
        else:
            stats[stat] = pd.to_numeric(stats[stat])
#                 print(stat + ': ')
#                 print(str(stats[stat].mean()))
            ballers[stat].append(stats[stat].mean())
            ballers[stat].append(stats[stat].mean())
            
    ballers['Photo'].append(soup.find_all('img', alt=("Photo of " + ballers['Name'][i]))[0]['src'])
    ballers['City'].append(soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " "))
    ballers['Country'].append(soup.find_all('span')[13].text)



# ballers.pop('Page')
# ballers.pop('Season')
# ballers.pop('Unnamed: 24')
# ballers.pop('Unnamed: 19')
# ballers


In [8]:
ballers

{'Name': ['Kostas Antetokounmpo',
  'Avery Bradley',
  'Devontae Cacok',
  'Kentavious Caldwell-Pope',
  'Alex Caruso',
  'Quinn Cook',
  'Troy Daniels',
  'Anthony Davis',
  'Jared Dudley',
  'Danny Green',
  'Talen Horton-Tucker',
  'Dwight Howard',
  'LeBron James',
  'Kyle Kuzma',
  'JaVale McGee',
  'Markieff Morris',
  'Zach Norvell',
  'Rajon Rondo',
  'J.R. Smith',
  'Dion Waiters'],
 'Page': ['https://www.basketball-reference.com//players/a/antetko01.html',
  'https://www.basketball-reference.com//players/b/bradlav01.html',
  'https://www.basketball-reference.com//players/c/cacokde01.html',
  'https://www.basketball-reference.com//players/c/caldwke01.html',
  'https://www.basketball-reference.com//players/c/carusal01.html',
  'https://www.basketball-reference.com//players/c/cookqu01.html',
  'https://www.basketball-reference.com//players/d/danietr01.html',
  'https://www.basketball-reference.com//players/d/davisan02.html',
  'https://www.basketball-reference.com//players/d/dud

In [191]:
stats

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,2015-16,27.0,NYK,NBA,PF,59.0,5.0,22.3,2.9,6.5,...,NaN,1.2,0.5,1.7,0.062,NaN,-1.7,-1.5,-3.2,-0.4
1,2016-17,28.0,NYK,NBA,PF,46.0,15.0,21.0,2.1,5.3,...,NaN,0.6,0.4,0.9,0.046,NaN,-2.4,-1.1,-3.5,-0.4
2,2017-18,29.0,NYK,NBA,PF,73.0,31.0,18.5,1.4,3.7,...,NaN,0.3,0.6,0.9,0.031,NaN,-3.2,-0.5,-3.7,-0.6
3,2018-19,30.0,NYK,NBA,PF,46.0,17.0,17.0,1.7,4.3,...,NaN,-0.3,0.3,0.1,0.004,NaN,-4.8,-0.9,-5.7,-0.7
4,2019-20,31.0,BRK,NBA,PF,7.0,4.0,14.0,1.1,3.3,...,NaN,-0.1,0.1,0.0,-0.015,NaN,-5.7,-2.6,-8.3,-0.2


In [162]:
baller_df = pd.DataFrame(ballers)
baller_df

,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Kostas Antetokounmpo,https://www.basketball-reference.com/req/20210...,"Athens, Greece",gr,22.0,LAL,NBA,PF,7.0,0.0,...,18.950000,16.400000,0.000000,0.000000,0.000000,-0.027000,-6.200000,-0.500000,-6.700000,0.000000
1,Avery Bradley,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,29.0,LAL,NBA,SG,834.0,825.0,...,12.295238,19.576190,-0.442857,0.923810,0.490476,0.020381,-2.490476,-0.728571,-3.223810,-0.361905
2,Devontae Cacok,https://www.basketball-reference.com/req/20210...,"Chicago, Illinois",us,23.0,LAL,NBA,C,1.0,0.0,...,0.000000,28.100000,0.000000,0.000000,0.000000,0.226000,4.800000,-4.600000,0.200000,0.000000
3,Kentavious Caldwell-Pope,https://www.basketball-reference.com/req/20210...,"Thomaston, Georgia",us,26.0,LAL,NBA,SG,377.0,274.0,...,8.840000,17.420000,2.280000,2.320000,4.560000,0.095600,-0.720000,0.420000,-0.300000,1.000000
4,Alex Caruso,https://www.basketball-reference.com/req/20210...,"College Station, Texas",us,25.0,LAL,NBA,PG,126.0,13.0,...,18.533333,15.233333,0.300000,1.000000,1.266667,0.076333,-2.466667,1.333333,-1.100000,0.200000
5,Quinn Cook,https://www.basketball-reference.com/req/20210...,"Washington, District of Columbia",us,26.0,LAL,NBA,PG,235.0,29.0,...,15.366667,19.691667,0.208333,0.191667,0.408333,0.045500,-1.591667,-1.600000,-3.200000,-0.058333
6,Troy Daniels,https://www.basketball-reference.com/req/20210...,"Roanoke, Virginia",us,28.0,DEN,NBA,SG,522.0,19.0,...,7.600000,17.476923,0.253846,0.330769,0.615385,0.040231,-1.792308,-1.623077,-3.407692,-0.069231
7,Anthony Davis,https://www.basketball-reference.com/req/20210...,"Chicago, Illinois",us,26.0,LAL,NBA,PF,329.0,329.0,...,8.980000,30.200000,6.440000,4.060000,10.500000,0.218600,5.160000,1.620000,6.780000,5.040000
8,Jared Dudley,https://www.basketball-reference.com/req/20210...,"San Diego, California",us,34.0,LAL,NBA,PF,297.0,74.0,...,14.420000,11.020000,1.020000,0.960000,1.960000,0.079800,-2.120000,0.880000,-1.240000,0.280000
9,Danny Green,https://www.basketball-reference.com/req/20210...,"North Babylon, New York",us,32.0,LAL,NBA,SG,365.0,355.0,...,11.680000,14.540000,0.900000,3.080000,3.960000,0.098200,-0.600000,1.540000,0.960000,1.460000


In [165]:
all_stats = pd.merge(baller_df, rank_df, on='Name')
all_stats = pd.merge(all_stats, percentile_df, on='Name')
all_stats

,Name,Photo,City,Country,Age,Tm,Lg,Pos,G,GS,...,%ile in TOV%,%ile in USG%,%ile in OWS,%ile in DWS,%ile in WS,%ile in WS/48,%ile in OBPM,%ile in DBPM,%ile in BPM,%ile in VORP
0,Kostas Antetokounmpo,https://www.basketball-reference.com/req/20210...,"Athens, Greece",gr,22.0,LAL,NBA,PF,7.0,0.0,...,90.0,25.0,15.0,0.0,5.0,5.0,5.0,35.0,5.0,25.0
1,Avery Bradley,https://www.basketball-reference.com/req/20210...,"Tacoma, Washington",us,29.0,LAL,NBA,SG,834.0,825.0,...,45.0,50.0,0.0,40.0,30.0,15.0,20.0,25.0,20.0,0.0
2,Devontae Cacok,https://www.basketball-reference.com/req/20210...,"Chicago, Illinois",us,23.0,LAL,NBA,C,1.0,0.0,...,0.0,85.0,15.0,0.0,5.0,95.0,85.0,0.0,75.0,25.0
3,Kentavious Caldwell-Pope,https://www.basketball-reference.com/req/20210...,"Thomaston, Georgia",us,26.0,LAL,NBA,SG,377.0,274.0,...,10.0,30.0,80.0,75.0,80.0,65.0,50.0,60.0,65.0,80.0
4,Alex Caruso,https://www.basketball-reference.com/req/20210...,"College Station, Texas",us,25.0,LAL,NBA,PG,126.0,13.0,...,85.0,15.0,45.0,55.0,45.0,55.0,25.0,75.0,50.0,50.0
5,Quinn Cook,https://www.basketball-reference.com/req/20210...,"Washington, District of Columbia",us,26.0,LAL,NBA,PG,235.0,29.0,...,70.0,55.0,35.0,15.0,20.0,30.0,45.0,10.0,25.0,20.0
6,Troy Daniels,https://www.basketball-reference.com/req/20210...,"Roanoke, Virginia",us,28.0,DEN,NBA,SG,522.0,19.0,...,5.0,35.0,40.0,25.0,35.0,25.0,40.0,5.0,15.0,5.0
7,Anthony Davis,https://www.basketball-reference.com/req/20210...,"Chicago, Illinois",us,26.0,LAL,NBA,PF,329.0,329.0,...,15.0,90.0,90.0,95.0,90.0,90.0,90.0,85.0,90.0,90.0
8,Jared Dudley,https://www.basketball-reference.com/req/20210...,"San Diego, California",us,34.0,LAL,NBA,PF,297.0,74.0,...,60.0,0.0,70.0,50.0,55.0,60.0,30.0,65.0,45.0,55.0
9,Danny Green,https://www.basketball-reference.com/req/20210...,"North Babylon, New York",us,32.0,LAL,NBA,SG,365.0,355.0,...,35.0,10.0,60.0,85.0,75.0,70.0,60.0,80.0,80.0,85.0


In [166]:
for col in all_stats.columns:
    print(col)

Name
Photo
City
Country
Age
Tm
Lg
Pos
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
eFG%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
PER
TS%
3PAr
FTr
ORB%
DRB%
TRB%
AST%
STL%
BLK%
TOV%
USG%
OWS
DWS
WS
WS/48
OBPM
DBPM
BPM
VORP
Rank in Age
Rank in G
Rank in GS
Rank in MP
Rank in FG
Rank in FGA
Rank in FG%
Rank in 3P
Rank in 3PA
Rank in 3P%
Rank in 2P
Rank in 2PA
Rank in 2P%
Rank in eFG%
Rank in FT
Rank in FTA
Rank in FT%
Rank in ORB
Rank in DRB
Rank in TRB
Rank in AST
Rank in STL
Rank in BLK
Rank in TOV
Rank in PF
Rank in PTS
Rank in PER
Rank in TS%
Rank in 3PAr
Rank in FTr
Rank in ORB%
Rank in DRB%
Rank in TRB%
Rank in AST%
Rank in STL%
Rank in BLK%
Rank in TOV%
Rank in USG%
Rank in OWS
Rank in DWS
Rank in WS
Rank in WS/48
Rank in OBPM
Rank in DBPM
Rank in BPM
Rank in VORP
%ile in Age
%ile in G
%ile in GS
%ile in MP
%ile in FG
%ile in FGA
%ile in FG%
%ile in 3P
%ile in 3PA
%ile in 3P%
%ile in 2P
%ile in 2PA
%ile in 2P%
%ile in eFG%
%ile in FT
%ile in FTA
%ile in FT%
%ile in ORB
%ile in DRB

In [ ]:
# STOP HERE!!!!

In [349]:
# for baller in ballers:
#     print(pd.read_html(baller)[0])

# setup scrape
# url = ballers['Page'][0]

url = 'https://www.basketball-reference.com/players/j/jamesle01.html'

html = requests.get(url)

# scrape
soup = bs(html.text, 'html.parser')
table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]

stats = pd.read_html(table)[0]

stats

advanced = stats.loc[(stats['Season'] == '2019-20') | 
          (stats['Season'] == '2018-19') |
          (stats['Season'] == '2017-18') | 
          (stats['Season'] == '2016-17') | 
          (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

stats = pd.read_html(url)[0]

standard = stats.loc[(stats['Season'] == '2019-20') | 
          (stats['Season'] == '2018-19') |
          (stats['Season'] == '2017-18') | 
          (stats['Season'] == '2016-17') | 
          (stats['Season'] == '2015-16') ]

stats = pd.merge(standard, advanced, on='Season')

this = {}

# this['Photo'] = soup.find_all('img', alt=("Photo of " + ballers['Name'][0]))[0]['src']

photo = soup.find_all('img', itemscope="image")[0]['src']
# [0]['src']
# city = soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " ")
# country = append(soup.find_all('span')[13].text

photo



ConnectionError: HTTPSConnectionPool(host='www.basketball-reference.com', port=443): Max retries exceeded with url: /players/j/jamesle01.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa820b8e310>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [210]:

stats["MP"] = pd.to_numeric(stats["MP"])
stats["MP"][0]
# stats["MP"].mean()


24.4

In [ ]:
kostas = {}

kostas['Name'] = ballers['Name'][0]

i = 0
for c in stats.columns:
    if i < 5:
        kostas[c] = stats[c][len(stats[c])-1]
    elif i < 7:
        kostas[c] = stats[c].sum()
    else:
        kostas[c] = stats[c].mean()
    i = i+1
    
kostas['Photo'] = ""

ballers['Name'][12]

# kostas.pop('Season')
# kostas.pop('Unnamed: 24')
# kostas.pop('Unnamed: 19')
# kostas

In [ ]:
# quit the browser
browser.quit()

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

# Jump to image link and setup soup
browser.links.find_by_partial_text('FULL IMAGE').click()
html = browser.html
soup = bs(html, 'html.parser')

# Get featured image url
featured_image_url = url.replace('index.html', soup.find_all('img')[1]['src'])

# quit the browser
browser.quit()


In [147]:
# Try to scrape the site using the URL
AD ='https://www.basketball-reference.com/players/d/davisan02.html'
LBJ = 'https://www.basketball-reference.com/players/j/jamesle01.html'
Kos = 'https://www.basketball-reference.com/players/a/antetko01.html'

url = AD

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get article title and summary paragraph

# city = soup.find_all('span', itemprop="birthPlace")[0].text
country = soup.find_all('span')[13].text
st = soup.find_all('table')[0]


city = soup.find_all('span', itemprop="birthPlace")[0].text.replace('\n    in\xa0', "").replace('\xa0', " ")
country = soup.find_all('span')[13].text

print(city)
print(country)


# news_H = soup.find_all('div', class_='content_title')[0].a.text.replace("\n", "")
# news_P = soup.find_all('div', class_='rollover_description_inner')[0].text.replace("\n", "")


Chicago, Illinois
us


In [ ]:
# Try to scrape the site using the URL
url = 'https://space-facts.com/mars/'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get Mars descriptions
col1 = soup.find_all('td', class_='column-1')
col2 = soup.find_all('td', class_='column-2')
props = []
stats = []

# Put descriptions in a dataframe
for x in range(0, len(col1)):
    props.append(col1[x].text)
    stats.append(col2[x].text)
    
fax_table = pd.DataFrame({"Description": props, "Mars": stats}).set_index('Description').to_html().replace("\n", "").replace("   ", " ")



In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')

# Get pages to jump to and set arrays to fill
jumps = []
titles = []
img_urls = []
hemisphere_urls = []

that = soup.find_all('h3')

for this in that:
    jumps.append(this.text)
    
# Jump through pages and get image titles and urls
for jump in jumps:
    try:
        browser.links.find_by_partial_text(jump).click()
        html = browser.html
        soup = bs(html, 'html.parser')
        titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
        img_urls.append(soup.find_all('a')[4]['href'])
        print("1 " + jump)
    except:
        try:
            browser.links.find_by_partial_text('2').click()
            browser.links.find_by_partial_text(jump).click()
            html = browser.html
            soup = bs(html, 'html.parser')
            titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
            img_urls.append(soup.find_all('a')[4]['href'])
            print("2 " + jump)
        except:
            browser.links.find_by_partial_text('1').click()
            browser.links.find_by_partial_text(jump).click()
            html = browser.html
            soup = bs(html, 'html.parser')
            titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
            img_urls.append(soup.find_all('a')[4]['href'])
            print("1 " + jump)

# Quit the browser
browser.quit()

# Set results to list of dictionaries
for x in range(0, len(img_urls)):
    hemisphere_urls.append({"title": titles[x], "img_url": img_urls[x]})


In [ ]:
mars = {"news_H": news_H,
        "news_P": news_P,
        "featured_image_url": featured_image_url,
        "fax_table": fax_table,
        "hemisphere_urls": hemisphere_urls}
mars
